To use this file, to prune a specific checkpoint for its MLP layers, replace ckpt and ckpt_path in cell 4 and use "Run all"

In [67]:
ls /net/vast-storage/scratch/vast/kellislab/artliang/magneton/finetuning/esmc_300m_active_ewc

checkpoints_esmc_300m_active_ewc/     run.sh              slurm-42279196.out
model_esmc_300m_active_ewc_no_mlp.pt  slurm-42279169.out
model_esmc_300m_active_ewc.pt         slurm-42279186.out


In [68]:
import torch
from magneton.config import PipelineConfig

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
ckpt = '/net/vast-storage/scratch/vast/kellislab/artliang/magneton/finetuning/esmc_300m_homo_ewc/model_esmc_300m_homo_ewc.pt'
ckpt_path = '/net/vast-storage/scratch/vast/kellislab/artliang/magneton/finetuning/esmc_300m_homo_ewc/model_esmc_300m_homo_ewc'

# special for ss ewc
# ckpt = '/net/vast-storage/scratch/vast/kellislab/artliang/magneton/finetuning/esmc_300m_ss_ewc/checkpoints_esmc_300m_ss_ewc/model_esmc_300m_ss_lr_default.ckpt'
# ckpt_path = '/net/vast-storage/scratch/vast/kellislab/artliang/magneton/finetuning/esmc_300m_ss_ewc/checkpoints_esmc_300m_ss_ewc/model_esmc_300m_ss_lr_default'

# .ckpt files are less complicated
# model = torch.load(ckpt, map_location=torch.device('cuda'))

# If you trust the checkpoint source
torch.serialization.add_safe_globals([
    PipelineConfig,
    # Add other classes if needed
])

model = torch.load(ckpt, map_location=torch.device('cuda'), weights_only=False)
# print(model)

In [82]:
# See what's in the model
print("Checkpoint keys:", model.keys())

# If it's a PyTorch Lightning model, the model is usually in 'state_dict'
if 'state_dict' in model:
    state_dict = model['state_dict']
    print("\nModel layers:")
    for name, param in state_dict.items():
        print(f"{name}: {param.shape}")

Checkpoint keys: dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

Model layers:
fisher_info: torch.Size([332997184])
embedder.model.embed.weight: torch.Size([64, 960])
embedder.model.transformer.blocks.0.attn.layernorm_qkv.0.weight: torch.Size([960])
embedder.model.transformer.blocks.0.attn.layernorm_qkv.0.bias: torch.Size([960])
embedder.model.transformer.blocks.0.attn.layernorm_qkv.1.weight: torch.Size([2880, 960])
embedder.model.transformer.blocks.0.attn.out_proj.weight: torch.Size([960, 960])
embedder.model.transformer.blocks.0.attn.q_ln.weight: torch.Size([960])
embedder.model.transformer.blocks.0.attn.k_ln.weight: torch.Size([960])
embedder.model.transformer.blocks.0.ffn.0.weight: torch.Size([960])
embedder.model.transformer.blocks.0.ffn.0.bias: torch.Size([960])
embedder.model.transformer.blocks.0.ffn.1.weight: torch.Size([5120, 960])
embedder.model.trans

In [83]:
# Create a new state_dict without MLP layers
new_state_dict = {}

for name, param in model['state_dict'].items():
    # Skip any parameter that contains 'mlp' in its name (and fisher in the case of EWC checkpoints)
    if 'mlp' not in name.lower() and 'fisher' not in name.lower():
        new_state_dict[name] = param
    else:
        print(f"Removing: {name}")

Removing: fisher_info
Removing: mlp.0.weight
Removing: mlp.0.bias
Removing: mlp.3.weight
Removing: mlp.3.bias
Removing: mlp.6.weight
Removing: mlp.6.bias
Removing: mlp.9.weight
Removing: mlp.9.bias


In [84]:
from esm.models.esmc import ESMC

esmc = ESMC.from_pretrained("esmc_300m")

In [85]:
base_model = esmc

base_keys = set(base_model.state_dict().keys())
print(base_keys)
checkpoint_keys = set(new_state_dict.keys())
print(checkpoint_keys)

print(f"Base model has {len(base_keys)} parameters")
print(f"Checkpoint has {len(checkpoint_keys)} parameters")

# Find mismatched keys
missing_in_checkpoint = base_keys - checkpoint_keys
extra_in_checkpoint = checkpoint_keys - base_keys

if missing_in_checkpoint:
    print(f"Missing in checkpoint ({len(missing_in_checkpoint)}):")
    for key in list(missing_in_checkpoint)[:5]:  # Show first 5
        print(f"  {key}")

if extra_in_checkpoint:
    print(f"Extra in checkpoint ({len(extra_in_checkpoint)}):")
    for key in list(extra_in_checkpoint)[:5]:  # Show first 5
        print(f"  {key}")

{'transformer.blocks.12.attn.k_ln.weight', 'transformer.blocks.14.ffn.0.bias', 'transformer.blocks.22.attn.k_ln.weight', 'transformer.blocks.16.attn.layernorm_qkv.0.bias', 'transformer.blocks.11.ffn.3.weight', 'transformer.blocks.9.ffn.3.weight', 'transformer.blocks.11.attn.q_ln.weight', 'transformer.blocks.26.ffn.0.bias', 'transformer.blocks.8.ffn.3.weight', 'transformer.blocks.23.attn.k_ln.weight', 'transformer.blocks.18.attn.out_proj.weight', 'transformer.blocks.15.attn.k_ln.weight', 'transformer.blocks.27.ffn.0.weight', 'transformer.blocks.29.ffn.3.weight', 'sequence_head.0.weight', 'transformer.blocks.1.attn.layernorm_qkv.0.bias', 'transformer.blocks.5.attn.out_proj.weight', 'transformer.blocks.3.attn.layernorm_qkv.1.weight', 'transformer.blocks.21.attn.layernorm_qkv.0.weight', 'transformer.blocks.26.ffn.1.weight', 'transformer.blocks.9.attn.q_ln.weight', 'transformer.blocks.18.attn.layernorm_qkv.0.weight', 'transformer.blocks.18.attn.q_ln.weight', 'transformer.blocks.25.attn.laye

In [86]:
# Remove prefix from checkpoint keys
prefix_state_dict = {}
prefix_to_remove = "embedder.model."  # adjust as needed
for key, value in new_state_dict.items():
    if key.startswith(prefix_to_remove):
        new_key = key[len(prefix_to_remove):]
        prefix_state_dict[new_key] = value
    else:
        prefix_state_dict[key] = value

In [87]:
prefix_keys = set(prefix_state_dict.keys())
print(prefix_keys)

{'transformer.blocks.12.attn.k_ln.weight', 'transformer.blocks.14.ffn.0.bias', 'transformer.blocks.22.attn.k_ln.weight', 'transformer.blocks.16.attn.layernorm_qkv.0.bias', 'transformer.blocks.11.ffn.3.weight', 'transformer.blocks.9.ffn.3.weight', 'transformer.blocks.11.attn.q_ln.weight', 'transformer.blocks.26.ffn.0.bias', 'transformer.blocks.8.ffn.3.weight', 'transformer.blocks.23.attn.k_ln.weight', 'transformer.blocks.18.attn.out_proj.weight', 'transformer.blocks.15.attn.k_ln.weight', 'transformer.blocks.27.ffn.0.weight', 'transformer.blocks.29.ffn.3.weight', 'sequence_head.0.weight', 'transformer.blocks.1.attn.layernorm_qkv.0.bias', 'transformer.blocks.5.attn.out_proj.weight', 'transformer.blocks.3.attn.layernorm_qkv.1.weight', 'transformer.blocks.21.attn.layernorm_qkv.0.weight', 'transformer.blocks.26.ffn.1.weight', 'transformer.blocks.9.attn.q_ln.weight', 'transformer.blocks.18.attn.layernorm_qkv.0.weight', 'transformer.blocks.18.attn.q_ln.weight', 'transformer.blocks.25.attn.laye

In [88]:
# Find mismatched keys
missing_in_checkpoint = base_keys - prefix_keys
extra_in_checkpoint = prefix_keys - base_keys

if missing_in_checkpoint:
    print(f"Missing in checkpoint ({len(missing_in_checkpoint)}):")
    for key in list(missing_in_checkpoint)[:5]:  # Show first 5
        print(f"  {key}")

if extra_in_checkpoint:
    print(f"Extra in checkpoint ({len(extra_in_checkpoint)}):")
    for key in list(extra_in_checkpoint)[:5]:  # Show first 5
        print(f"  {key}")

In [89]:
base_model.load_state_dict(prefix_state_dict, strict=True)

<All keys matched successfully>

In [91]:
# Create new model with the filtered state_dict
new_model = model.copy()  # Keep other metadata
new_model['state_dict'] = prefix_state_dict

# Save the new model
# new_ckpt_path = '/weka/scratch/weka/kellislab/rcalef/projects/magneton/experiments/downstream_evals/deepfri_esmc_300m_domain_default_lr/EC/epoch=0-valid_auprc=0.02_no_mlp.ckpt'
new_ckpt_path = f'{ckpt_path}_no_mlp.ckpt'
torch.save(new_model, new_ckpt_path)

print(f"\nOriginal model had {len(model['state_dict'])} parameters")
print(f"New model has {len(prefix_state_dict)} parameters")
print(f"Removed {len(model['state_dict']) - len(prefix_state_dict)} MLP parameters")
print(f"Saved to: {new_ckpt_path}")


Original model had 317 parameters
New model has 308 parameters
Removed 9 MLP parameters
Saved to: /net/vast-storage/scratch/vast/kellislab/artliang/magneton/finetuning/esmc_300m_ss_ewc/checkpoints_esmc_300m_ss_ewc/model_esmc_300m_ss_lr_default_no_mlp.ckpt
